# 如何使用 SmartLLMChain

SmartLLMChain 是一种自我批评链，可以帮助你解决特别复杂的问题。它不是单次 LLM 调用，而是执行以下 3 个步骤：
1. 构思：将用户提示传递给 LLM n 次以获得 n 个输出提案（称为"想法"），其中 n 是你可以设置的参数
2. 批评：LLM 对所有想法进行批评以找出可能的缺陷并选择最佳方案
3. 解决：LLM 试图改进最佳想法（在批评步骤中选择的）并输出它。这就是最终输出。

SmartLLMChains 基于 https://youtu.be/wVzuvf9D9BU 中提出的 SmartGPT 工作流。

请注意 SmartLLMChains
- 使用更多的 LLM 调用（即 n+2 而不是仅 1 次）
- 只有在底层 LLM 具有反思能力时才能工作，这是较小模型通常不具备的
- 只能与返回恰好 1 个输出而不是多个输出的底层模型一起工作

本笔记本演示了如何使用 SmartLLMChain。

##### 所有步骤使用相同的 LLM

In [1]:
import os

os.environ["OPENAI_API_KEY"] = "..."

In [2]:
from langchain.prompts import PromptTemplate
from langchain_experimental.smart_llm import SmartLLMChain
from langchain_openai import ChatOpenAI

作为示例问题，我们将使用"我有一个 12 升的壶和一个 6 升的壶。我想要测量 6 升水。我该怎么做？"这是来自原始 SmartGPT 视频的例子（https://youtu.be/wVzuvf9D9BU?t=384）。虽然这看起来是一个很简单的问题，但 LLM 在处理这类涉及数字和物理推理的问题时会遇到困难。

正如我们将看到的，所有 3 个初始想法都是完全错误的 - 尽管我们使用的是 GPT4！只有在使用自我反思时，我们才能得到正确的答案。

In [ ]:
hard_question = "我有一个 12 升的壶和一个 6 升的壶。我想要测量 6 升水。我该怎么做？"

首先，我们创建一个 LLM 和提示模板

In [3]:
prompt = PromptTemplate.from_template(hard_question)
llm = ChatOpenAI(temperature=0, model_name="gpt-4")

现在我们可以创建一个 SmartLLMChain

In [4]:
chain = SmartLLMChain(llm=llm, prompt=prompt, n_ideas=3, verbose=True)

现在我们可以将 SmartLLM 作为 LLM 的即插即用替代品。例如：

In [5]:
chain.invoke({})



> Entering new SmartLLMChain chain...
Prompt after formatting:
I have a 12 liter jug and a 6 liter jug. I want to measure 6 liters. How do I do it?
Idea 1:
1. Fill the 6-liter jug completely.
2. Pour the water from the 6-liter jug into the 12-liter jug.
3. Fill the 6-liter jug again.
4. Carefully pour the water from the 6-liter jug into the 12-liter jug until the 12-liter jug is full.
5. The amount of water left in the 6-liter jug will be exactly 6 liters.
Idea 2:
1. Fill the 6-liter jug completely.
2. Pour the water from the 6-liter jug into the 12-liter jug.
3. Fill the 6-liter jug again.
4. Carefully pour the water from the 6-liter jug into the 12-liter jug until the 12-liter jug is full.
5. Since the 12-liter jug is now full, there will be 2 liters of water left in the 6-liter jug.
6. Empty the 12-liter jug.
7. Pour the 2 liters of water from the 6-liter jug into the 12-liter jug.
8. Fill the 6-liter jug completely again.
9. Pour the water from the 6-liter jug into the 12-liter j

'1. Fill the 12-liter jug completely.\n2. Pour the water from the 12-liter jug into the 6-liter jug until the 6-liter jug is full.\n3. The amount of water left in the 12-liter jug will be exactly 6 liters.'

##### 不同步骤使用不同的 LLM

你也可以通过传递 `ideation_llm`、`critique_llm` 和 `resolve_llm` 为不同的步骤使用不同的 LLM。你可能想要这样做是为了在构思阶段使用更具创造性的（即高温度）模型，而在批评和解决阶段使用更严格的（即低温度）模型。

In [ ]:
chain = SmartLLMChain(
    ideation_llm=ChatOpenAI(temperature=0.9, model_name="gpt-4"),
    llm=ChatOpenAI(
        temperature=0, model_name="gpt-4"
    ),  # 将用于批评和解决，因为没有给出特定的 llms
    prompt=prompt,
    n_ideas=3,
    verbose=True,
)